In [ ]:
%matplotlib inline

## YOLO

In [ ]:
from src.hpe.yolo.performance import read_distances

distances = read_distances()

In [ ]:
from src.hpe.yolo.plot import plot_yolo_average_distances

plot_yolo_average_distances(distances=distances)

## MediaPipe

In [ ]:
from src.hpe.mp.performance import read_distances

distances_mp = read_distances()

In [ ]:
from src.hpe.mp.plot import plot_mediapipe_average_distances

plot_mediapipe_average_distances(distances=distances_mp)

## Comparison